In [1]:
import pandas as pd

data = pd.read_csv("LOCAL_PEOPLE_DONG_202110.csv",encoding='UTF-8')
#data.columns

#변수 추출
data=data[['기준일ID', '시간대구분', '행정동코드', '남자20세부터24세생활인구수', '남자25세부터29세생활인구수',
           '여자20세부터24세생활인구수','여자25세부터29세생활인구수']]

#변수 이름 변경
data.rename(columns={'기준일ID':'date', '시간대구분':'time', '행정동코드':'adm_cd', '남자20세부터24세생활인구수':'m_20', '남자25세부터29세생활인구수':'m_25',
           '여자20세부터24세생활인구수':'f_20','여자25세부터29세생활인구수':'f_25'},inplace=True)

#20대 변수 생성
data['age_20']=data['m_20']+data['m_25']+data['f_20']+data['f_25']


#서초구 뽑아내기
data_sc = data[(data['adm_cd']>=11650510) & (data['adm_cd']<=11650660)]


#10월 25일부터 10월 31일까지 데이터 생성
data_sc = data_sc[(data_sc['date']>=20211025)&(data_sc['date']<=20211031)]

#서초구 데이터 변수 선택

data_sc=data_sc[['date','time','adm_cd','age_20']]




In [2]:
#date 요일로 범주 변경
recode_date={'date':{20211025:'월요일',20211026:'화요일',20211027:'수요일',20211028:'목요일',
            20211029:'금요일',20211030:'토요일',20211031:'일요일'}}
data_sc=data_sc.replace(recode_date)

# time 시로 범주 변경
recode_time={'time':{0:"00시",1:"01시",2:"02시",3:"03시",4:"04시",5:"05시",6:"06시",7:"07시",8:"08시",9:"09시",
            10:"10시",11:"11시",12:"12시",13:"13시",14:"14시",15:"15시",16:"16시",17:"17시",18:"18시",19:"19시",
            20:"20시",21:"21시",22:"22시",23:"23시"}}
data_sc=data_sc.replace(recode_time)
data_sc

,date,time,adm_cd,age_20
244517,월요일,00시,11650580,1706.9932
244549,월요일,00시,11650570,1208.1928
244563,월요일,00시,11650560,4454.1030
244570,월요일,00시,11650510,3439.2113
244571,월요일,00시,11650550,381.3074
...,...,...,...,...
315404,일요일,23시,11650570,1296.3799
315406,일요일,23시,11650550,395.4483
315410,일요일,23시,11650581,3317.1897
315425,일요일,23시,11650531,3676.8976


In [47]:
#요일별 행정동 통계
sc_date = data_sc.groupby(['date','adm_cd']).mean()
sc_date = sc_date.reset_index(drop=False)
sc_date

,date,adm_cd,age_20
0,금요일,11650510,5040.681483
1,금요일,11650520,7957.995221
2,금요일,11650530,11075.473617
3,금요일,11650531,8263.539150
4,금요일,11650540,4368.338871
...,...,...,...
121,화요일,11650620,2396.645425
122,화요일,11650621,3318.542638
123,화요일,11650651,6987.962492
124,화요일,11650652,5513.318671


In [8]:
#시간별 행정동 통계
sc_time = data_sc.groupby(['time','adm_cd']).mean()
sc_time = sc_time.reset_index(drop=False)
sc_time

,time,adm_cd,age_20
0,00시,11650510,3598.480329
1,00시,11650520,4519.255114
2,00시,11650530,7324.835614
3,00시,11650531,3418.567214
4,00시,11650540,3272.118214
...,...,...,...
427,23시,11650620,2538.058657
428,23시,11650621,3146.012957
429,23시,11650651,4847.923643
430,23시,11650652,3976.274114


In [27]:
# 서초구 14시 데이터 추출
#멀티 인덱싱 sc_time_14=sc_time.loc[('14시',)]
sc_time_14 = sc_time[sc_time['time']=='14시']

sc_time_14.adm_cd = sc_time_14.adm_cd.astype(str)

C:\Users\LG\anaconda3\envs\pydatavenv\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [48]:
#서초구 금요일 데이터 추출
sc_date_f=sc_date[sc_date['date']=='금요일']
sc_date_f.adm_cd = sc_date_f.adm_cd.astype(str)

C:\Users\LG\anaconda3\envs\pydatavenv\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [28]:
import json
seocho_geo='seocho1.geojson'
#geo_data=json.load(open(seocho_geo,'rt',encoding='UTF8'))
geo_data=json.load(open(seocho_geo,'rt'))
#print(geo_data)

In [54]:
#서초구 14시 지도 만들기
import folium
#geo_data='서울_행정동_경계_2017.json'

center = [37.473552,127.030250]

seoul_map = folium.Map(location=center,zoom_strat=15)


folium.Choropleth(geo_data=geo_data,
                 data = sc_time_14,
                 columns=('adm_cd','age_20'),
                 key_on='feature.properties.adm_cd',
                 fill_color='YlGnBu',
                 lengend_name="age_20",).add_to(seoul_map)

seoul_map

In [55]:
import folium
#geo_data='서울_행정동_경계_2017.json'

center = [37.473552,127.030250]

seoul_map = folium.Map(location=center,zoom_strat=15)


folium.Choropleth(geo_data=geo_data,
                 data = sc_date_f,
                 columns=('adm_cd','age_20'),
                 key_on='feature.properties.adm_cd',
                 fill_color='YlGnBu',
                 lengend_name="age_20",).add_to(seoul_map)

seoul_map